# Исследование надежности заемщиков


Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Описание данных**

`children` — количество детей в семье

`days_employed` — общий трудовой стаж в днях

`dob_years` — возраст клиента в годах

`education` — уровень образования клиента

`education_id` — идентификатор уровня образования

`family_status` — семейное положение

`family_status_id` — идентификатор семейного положения

`gender` — пол клиента

`income_type` — тип занятости

`debt` — имел ли задолженность по возврату кредитов

`total_income` — ежемесячный доход

`purpose` — цель получения кредита

## Загрузка данных

In [ ]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`.  Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В данных встречаются аномалии, например отрицательное количество дней трудового стажа в столбце `days_employed`. Заменим все отрицательные значения положительными с помощью метода `abs()`.

In [6]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведим медианное значение трудового стажа `days_employed` в днях.

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится нам для исследования.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения.

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру. 

In [14]:
data['education'] = data['education'].str.lower()

Удалим строки дубликаты.

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, созддим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.



In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.


In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для начала посчитаем количество строк в таблице для каждого уникального значения количества детей:

In [22]:
data['children'].value_counts()

0    14091
1     4808
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

Как видно из результатов, количество детей у клиентов варьируется от 0 до 5 детей, при этом чем больше детей, тем меньше таких клиентов встречается в таблице. Примерно 14000 клиентов в таблице не имеют детей и всего 9 клиентов имеют по 5 детей.

Найдем теперь сколько из этих клиентов банка не имели задолжностей по возврату кредита (значение в столбце `debt=0`) и посчитаем процентное отношение таких клиентов к общему количеству клиентов для каждой категории количества детей:

In [23]:
data.loc[data['debt']==0]['children'].value_counts()/data['children'].value_counts()*100

0     92.456178
1     90.765391
2     90.545809
3     91.818182
4     90.243902
5    100.000000
Name: children, dtype: float64

Из результата видно, что только клиенты, у которых по 5 детей, не имели никаких задолженностей по кредиту, однако небольшое количество таких клиентов из общей выборки не позволяет судить однозначно: случайность это или закономернность. При этом клиенты с количеством детей от 1 до 4 чаще имеют задолженности по сравнению с клиентами без детей. Для того тобы убедится в этом окончательно, разобьем всех клиентов на категории `'есть дети'` и `'нет детей'` и проведем аналогичные расчеты:  

In [24]:
def categorize_children(children):
    try:
        if children == 0:
            return 'нет детей'
        elif children >0:
            return 'есть дети'
    except:
        pass
    
#добавим столбец 'children_category' в данные     
data['children_category'] = data['children'].apply(categorize_children)

In [25]:
data['children_category'].value_counts()

нет детей    14091
есть дети     7240
Name: children_category, dtype: int64

Теперь мы видим что у нас примерно 2/3 всех клиентов не имеют детей и 1/3 имеет 1 и более детей,эти две категории уже достаточно большие чтобы их можно было сравнивать между собой.

In [26]:
#посчитаем отношение в процентах и округлим результат до десятых
round(data.loc[data['debt']==0]['children_category'].value_counts()/data['children_category'].value_counts()*100, ndigits=1)

нет детей    92.5
есть дети    90.8
Name: children_category, dtype: float64

In [27]:
#функция отношения количества вклиентов имеющих задолженностей к общему количеству клиентов
def ratio_delay(x):
    return '{:.2%}'.format(sum(x)/len(x)) 

#построим сводную таблицу количества клиентов, количества просрочек и доли просрочек по группам.
pd.pivot_table(data,
               index=['children_category'],
               values=['debt'],
               aggfunc=[len, sum, ratio_delay])

,len,sum,ratio_delay
,debt,debt,debt
children_category,,,
есть дети,7240,669,9.24%
нет детей,14091,1063,7.54%


Таким образом результаты анализа таблицы показывают, что клиенты имеющиие 1 и более детей в среднем чаще имеют задолженности по кредиту по сравнению с клиентами без детей.

**Вывод:** 

Зависимость между наличием детей и возвратом кредита в срок есть. У клиентов с 1 и более детей чаще имеются задолженности по кредиту, по сравнению с клиентами без детей. Более точной зависимости от количества детей установить сложно, поскольку в категориях с многодетными семьями количество клиентов недостаточно для формирования однозначных выводов.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

Проанализируем данные с точки зрения семейного положения. Для начала посмотрим какое количество клиентов находятся в каждой из категорий семейного положения:

In [28]:
data.loc[data['debt']==0]['family_status'].value_counts()

женат / замужем          11334
гражданский брак          3749
Не женат / не замужем     2523
в разводе                 1105
вдовец / вдова             888
Name: family_status, dtype: int64

Видно, что наибольшую категорию составляют клиенты находящиеся в законном браке и меньше всего вдов/вдовцов. При этом нет такого экстремального дисбаланса классов как в случае с разбивкой по количеству детей.

Построим сводную таблицу количества клиентов, количества просрочек и доли просрочек по категориям.

In [29]:
pd.pivot_table(data,
               index=['family_status'],
               values=['debt'],
               aggfunc=[len, sum, ratio_delay])

,len,sum,ratio_delay
,debt,debt,debt
family_status,,,
Не женат / не замужем,2796,273,9.76%
в разводе,1189,84,7.06%
вдовец / вдова,951,63,6.62%
гражданский брак,4134,385,9.31%
женат / замужем,12261,927,7.56%


Из анализа результатов видно, что самыми надежными заемщиками являются категории клиентов бывших в браке или состоящих в нем (`'женат / замужем', 'в разводе', 'вдовец / вдова'`), самыми ненадежными являются клиенты не имеющие зарегестрированных отношений (`'Не женат / не замужем', 'гражданский брак'`).

Попробуем посмотреть зависят ли такие результаты от наличия/отсутствия детей.

In [30]:
pd.pivot_table(data,
               index=['family_status'],
               columns=['children_category'],
               values=['debt'],
               aggfunc=[len, sum, ratio_delay])

len                 sum           ratio_delay  \
                           debt                debt                  debt   
children_category     есть дети нет детей есть дети нет детей   есть дети   
family_status                                                               
Не женат / не замужем       534      2262        63       210      11.80%   
в разводе                   405       784        29        55       7.16%   
вдовец / вдова              104       847        10        53       9.62%   
гражданский брак           1404      2730       156       229      11.11%   
женат / замужем            4793      7468       411       516       8.58%   

                                 
                                 
children_category     нет детей  
family_status                    
Не женат / не замужем     9.28%  
в разводе                 7.02%  
вдовец / вдова            6.26%  
гражданский брак          8.39%  
женат / замужем           6.91%

При разделении на подкатегории появились некоторые отличия. По прежнему самой ненадежной категорией остается "не женат/незамужем" вне зависимости от наличия детей. А вот самая отвественная в плане выплаты кредита, категория 'вдовец/вдова' сильно зависит от наличия детей: 6.26% при остуствии детей и 9.62% при наличии детей. При таком рассмотрении категория клиентов в гражданском браке и без детей является более надежной чем вдовец/вдова с детьми (8.39% против 9.62%)

**Вывод:** 

Зависимость между семейным положением и возвратом кредита в срок есть. Были проанализированы как средние общие показатели числа клиентов не имеющих задолженностей, так и в разбивке на подкатегории в зависимости от наличия детей. Самые частые случаи невозврата кредита в срок наблюдаются у клиентов категории "Не женат / не замужем" в независимости от наличия детей. Самые редкие случаи невозврата кредита в срок наблюдаются у клиентов категории "вдовец / вдова" в случае отсутстви детей у клиентов и категории "в разводе" в случае наличия детей. В целом наблюдается тенденция более частых пробелем с выплатой кредита в срок у клиентов не состоящих или не состоявших ранее в законном браке, а также у клиентов имеющих детей.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Проанализируем данные с точки зрения уровня дохода. Для этого данные заранее были категоризированы по следующему правилу:
- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

Сперва проверим сколько у нас клиентов по каждой из категорий выше:

In [31]:
data['total_income_category'].value_counts().sort_index(ascending=False) # выведем категории в порядке возрастания дохода


E       22
D      349
C    15921
B     5014
A       25
Name: total_income_category, dtype: int64

Видно, что данные по категориям не сбалансированы, категории клиентов с самыми низкими и самыми большими доходами являются малочисленными (22-25 человек), в то время как основная часть клиентов имеет категорию дохода С (почти 16 тысяч клиентов). основываясь на таком распределение количества клиентов по категориям, можно сказать, что категория C - это клиенты со средним уровнем дохода, D и B - клиенты с уровнем дохода ниже и выше среднего, соответственно, а клиенты категорий E и A - это клиенты с очень низким и очень высоким уровнем дохода.

Проверим сколько процентов клиентов в каждой из категорий имели задолженности по кредиту:

In [32]:
pd.pivot_table(data,
               index=['total_income_category'],
               values=['debt'],
               aggfunc=[len, sum, ratio_delay])

,len,sum,ratio_delay
,debt,debt,debt
total_income_category,,,
A,25,2,8.00%
B,5014,354,7.06%
C,15921,1353,8.50%
D,349,21,6.02%
E,22,2,9.09%


Результаты не выявляют какой либо прямой зависимости количества должников по кредиту от уровня дохода. Так клиенты с высоким и очень высоким уровнем дохода не гарантируют банку отсутствие задолженностей по кредиту. А клиенты с очень низким уровнем дохода, хоть и отражают наибольший процент в данной статистике, не сильно отличаются от клиентов со средним уровнем дохода. В этой статистике выделеяется немногочисленная категория клиентов с уровнем дохода ниже среднего, показывая самый низкий процент клиентов с задолженностями, однако учитывая количество клиентов в этой категории, сделать однозначный вывод о достоверности такого результата нельзя.

Попробуем рассмотреть влияние уровня дохода в совокупности с факторами семейного положения и наличия детей.

Для исследования уже 3х параметров объеденим категории по детям и семейному положению следующим образом:

- `'был/есть в официальном браке, без детей'` - если children=0, а family_status соответствует `'женат / замужем', 'в разводе', 'вдовец / вдова'`;
- `'не был в официальном браке, без детей'` - если children=0, а family_status соответствует `'Не женат / не замужем', 'гражданский брак'`;
- `'был/есть в официальном браке, есть дети'` - если children>0, а family_status соответствует `'женат / замужем', 'в разводе', 'вдовец / вдова'`;
- `'не был в официальном браке, есть дети'` - если children>0, а family_status соответствует `'Не женат / не замужем', 'гражданский брак'`;


In [33]:
def categorize_children(row):
    try:
        if row['children'] == 0:
            if row['family_status_id'] in [0,2,3]:
                return 'был/есть в официальном браке, без детей'
            else:
                return 'не был в официальном браке, без детей'
        elif row['children'] >0:
            if row['family_status_id'] in [0,2,3]:
                return 'был/есть в официальном браке, есть дети'
            else:
                return 'не был в официальном браке, есть дети'
    except:
        pass
    
#добавим столбец 'family_category' в данные     
data['family_category'] = data.apply(categorize_children, axis=1)

Построим сводные таблицы. Сначала для количества клиентов в каждой категории:

In [34]:
pd.pivot_table(data,
               index=['family_category'],
               columns=['total_income_category'],
               values=['debt'],
               aggfunc=len,
               fill_value=0)

debt                     
total_income_category                      A     B     C    D   E
family_category                                                  
был/есть в официальном браке, без детей    8  2067  6837  176  11
был/есть в официальном браке, есть дети    9  1307  3897   86   3
не был в официальном браке, без детей      4  1179  3736   67   6
не был в официальном браке, есть дети      4   461  1451   20   2

Теперь для доли клиентов не имеющих задолженности по кредиту:

In [35]:
ratio = lambda x: sum(x)/len(x)
pd.pivot_table(data,
               index=['family_category'],
               columns=['total_income_category'],
               values=['debt'],
               aggfunc=ratio).style.format('{:.2%}').background_gradient(cmap='Reds',axis=None)

При таком анализе данных вывляются интересные особенности для категорий с очень высоким (А) и очень низким (Е) уровнями дохода.
Несмотря на сделанные в предыдущем разделе выводы, о том, что самой благонадежной категорией заемщиков является категория  `был/есть в официальном браке, без детей`, для клиентов с очень низким уровнем дохода (Е) ситуация в точности наоборот. А для категории клиентов с очень высоким уровнем дохода (А) самыми ненадежными являются клиенты не состоящие в официальном браке без детей. Правда стоит заметить что такой высокий процент является следствием одного должника из всего четырех клиентов этой категории, что недостаточно для формирования однозначных выводов.

Остальные категории по доходам B, C и D демонстрируют те же зависимости, что были выявлены на предыдущих шагах исследования.

**Вывод:** 

Неравномерное распределение выборок по категориям не позволяет делать выводы по всем группам доходности клиентов. Самыми многочисленными являются группы клиентов со средним уровнем доходов (50001–200000) и уровнем доходов выше среднего (200001–1000000). Если использовать две эти группы для сравнения,то можно сделать выводы, что доля клиентов с просрочками уменьшается при возрастании уровня доходов клиента. Такая тенденция сохраняется при разбивке на подкатегории в зависимости от семейного положения и наличия детей.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

Сгруппируем клиентов по категориям, отражающим цели кредитаю Посчитаем распределение клиентов по категориям:

In [36]:
data['purpose_category'].value_counts()

операции с недвижимостью    10751
операции с автомобилем       4279
получение образования        3988
проведение свадьбы           2313
Name: purpose_category, dtype: int64

Видно, что операции с недвижимостью являются наиболее частой целью получения кредита.

Посчитаем процент клиентов c задолженностями по категориям:

In [37]:
pd.pivot_table(data,
               index=['purpose_category'],
               values=['debt'],
               aggfunc=[len, sum, ratio_delay])

,len,sum,ratio_delay
,debt,debt,debt
purpose_category,,,
операции с автомобилем,4279,400,9.35%
операции с недвижимостью,10751,780,7.26%
получение образования,3988,369,9.25%
проведение свадьбы,2313,183,7.91%


По результатам видно, что если целью кредита являются операции с недвижимостью, то процент клиентов с задолженностями минимальный. На втором месте стоит категория кредитов на проведение свадьбы. Одинаково часто клиенты имели задолженности, если целью кредита были операции с автомобилем и получение образования, в этих категориях процент клиентов с задолженностями наибольший.

Проверим сохраняется ли такое положение дел при различных уровнях дохода и семейного статуса.

Таблица зависимости процента клиентов с задолженностями для разных категорий дохода и целей кредита:

In [38]:
pd.pivot_table(data,
               index=['purpose_category'],
               columns=['total_income_category'],
               values=['debt'],
               aggfunc=ratio).style.format('{:.2%}').background_gradient(cmap='Reds',axis=None)

Если рассматривать зависимости от целей кредита для каждой категории уровня дохода отдельно, можно сделать следующие выводы:
- для категории с уровнем дохода ниже среднего (D) меньше всего задолженностей у клиентов с кредитами на операции с недвижимостью и получения образования, больше всего на операции с автомобилем;
- для категории с средним уровнем дохода (С) результаты соответствуют общей статистике выше;
- для категории с уровнем дохода выше среднего (В) меньше всего задолженностей у клиентов с кредитами на проведение свадьбы, затем идут операции с недвижимостью, а больше всего задолженностей по категории операций с автомобилями. 

Клиентов категорий D и E не принимаем в расчет, за недостаточностью данных.

Таблица зависимости процента клиентов без задолженностей для разных категорий семейного положения и целей кредита:

In [39]:
pd.pivot_table(data,
               index=['purpose_category'],
               columns=['family_category'],
               values=['debt'],
               aggfunc=ratio).style.format('{:.2%}').background_gradient(cmap='Reds',axis=None)

Если рассматривать зависимости от целей кредита для каждой категории семейного положения отдельно, то зависимости соответствуют общей статистике, за тем исключением что в категориях клиентов `'был/есть в официальном браке'` отсутствуют клиенты с кредитами на проведение свадьбы. 

**Вывод:** 

Влияние цели кредита на его возврат в срок по данным выявляется. Чаще всего не имеют задолженностей клиенты берущие кредит на операции с недвижимостью, на втором месте идет кредит на проведение свадьбы, а клиенты берущие кредиты на операции с автомобилем и получение образования в равной степени и чаще чем остальные категории не возвращают в срок. Такие же зависимости наблюдаются для любого семейного статуса и количества детей. А для клиентов с разным уровнем дохода зависимость возврата кредита в срок от цели кредита может слегка меняться.

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:*  В исходных данных были пропуски в столбцах days_employed и total_income, при чем если данных не было в одном столбце то их не было и в другом столбце. Причиной отсутствия данных может быть как ошибка технического характера (сбои в программе, ошибки при записи), так и не представление этих данных банку при заполнении заявки на кредит. Однако, учитывая что такая информация обычно является обязательной для одобрения кредита, скорее всего данные были каким то образом утеряны.

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* При наличии экстремальных значений в выборке (а в нашей таблице такие наблюдались - это категори A и E для уровня дохода), медианное значение позволяет более точно интерпретировать статистику, не беря в расчет экстремальные значения. 

### Шаг 4: общий вывод.

Результаты анализа данных статистики о платёжеспособности клиентов показали,что:

1. У клиентов с 1 и более детей чаще имеются задолженности по кредиту (9.24%) , по сравнению с клиентами без детей (7.54%). . Более точной зависимости от количества детей установить сложно, поскольку в категориях с многодетными семьями количество клиентов недостаточно для формирования однозначных выводов.

    
2. В целом наблюдается тенденция более частых проблем с выплатой кредита в срок у клиентов не состоящих или не состоявших ранее в законном браке.

    Самые частые случаи невозврата кредита в срок наблюдаются у клиентов категории "Не женат / не замужем" (9.76%) и "гражданский брак" (9.31%). Самые редкие случаи невозврата кредита в срок наблюдаются у клиентов категории "вдовец / вдова" (6.62%), категории "в разводе" (7.06%) и "женат / замужем" (7.56%). 

    При рассмотрении этих категорий с точки зрения отсутствия наличия детей имеются некоторые отличия. По прежнему самой ненадежной категорией остается "не женат/незамужем" вне зависимости от наличия детей. А вот самая отвественная в плане выплаты кредита, категория 'вдовец/вдова' сильно зависит от наличия детей: 6.26% при остуствии детей и 9.62% при наличии детей. При таком рассмотрении категория клиентов в гражданском браке и без детей является более надежной чем вдовец/вдова с детьми (8.39% против 9.62%).


3. Неравномерное распределение выборок по категориям не позволяет делать выводы по всем группам доходности клиентов. Группы клиентов с очень высоким уровнем доходов (1000001 и выше), очень низким уровнем доходов (0–30000) и доходами ниже среднего (30001–50000) являются наиболее редкими случаями в исходных данных и их не нельзя учитывать при выявлении зависимостей. Самыми многочисленными являются группы клиентов со средним уровнем доходов (50001–200000) и уровнем доходов выше среднего (200001–1000000). Если использовать две эти группы для сравнения,то можно сделать выводы, что доля клиентов с просрочками уменьшается при возрастании уровня доходов клиента (8.50% - для среднего уровня дохода, 7.06% - для уровня дохода выше среднего). Такая тенденция сохраняется при разбивке на подкатегории в зависимости от семейного положения и наличия детей.

    
4. Реже всего имеют задолженности клиенты берущие кредит на операции с недвижимостью (7.26%), на втором месте идет кредит на проведение свадьбы (7.91%), а клиенты берущие кредиты на операции с автомобилем и получение образования в равной степени и чаще чем остальные категории не возвращают в срок (9.35% и 9.25%, соответственно).
    
    Такие же зависимости наблюдаются для любого семейного статуса и количества детей. А для клиентов с разным уровнем дохода зависимость возврата кредита в срок от цели кредита может слегка меняться. Так для категории клиентов с уровнем дохода выше среднего самой надежной является группа клиентов берущих кредит на проведение свадьбы (5.15%), а затем идут операции с недвижимостью (6.72%). А для клиентов со средним уровнем дохода самой ненадежной стала группа клиентов берущих кредит на образование (9.85%), а после низ идут операции с автомобилем (9.59%).